In [43]:
import os
import json
import pandas as pd
import spacy


nlp = spacy.load("en_core_web_lg")

# Data Preparation

In [44]:
data_file = open('../Data/citation_data.json')

lines = data_file.readlines()
json_lines = []


In [45]:
for line in lines:
    json_lines.append(json.loads(line))

cc_df = pd.DataFrame(json_lines)

In [46]:
cc_df.head()

,src_paper_id,tgt_paper_id,src_abstract,tgt_abstract,explicit_citation,text_before_explicit_citation,text_after_explicit_citation,implicit_citation_0.1,text_before_implicit_citation_0.1,text_after_implicit_citation_0.1,implicit_citation_0.9,text_before_implicit_citation_0.9,text_after_implicit_citation_0.9,human_labeled,train_or_test
0,D09-1141,A00-1002,We propose a novel language-independent approa...,Using examples of the transfer-based MT system...,Other important publications about pivoting ap...,"For example, using German as a pivot, they ext...","Unlike pivoting, which can only improve source...",Other important publications about pivoting ap...,"For example, using German as a pivot, they ext...","Second, while pivoting ignores context when ex...",Other important publications about pivoting ap...,"For example, using German as a pivot, they ext...","Second, while pivoting ignores context when ex...",0,train
1,D12-1027,A00-1002,"We propose a novel, language-independent appro...",Using examples of the transfer-based MT system...,This has been tried for a number of language p...,One relevant line of research is on machine tr...,"Crimean Tatar #OTHEREFR, Irish. Scottish Gaeli...",This has been tried for a number of language p...,One relevant line of research is on machine tr...,"Crimean Tatar #OTHEREFR, Irish. Scottish Gaeli...",This has been tried for a number of language p...,One relevant line of research is on machine tr...,"Crimean Tatar #OTHEREFR, Irish. Scottish Gaeli...",0,train
2,H05-1110,A00-1002,Dictionaries and word translation models are u...,Using examples of the transfer-based MT system...,It is also worth noting that there has been so...,"They use a variety of approaches #OTHEREFR), m...",4Much of recent MT research focuses on pairs o...,"They do not, however, focus on related languag...",Many machine translation techniques require di...,4Much of recent MT research focuses on pairs o...,It is also worth noting that there has been so...,"They use a variety of approaches #OTHEREFR), m...",4Much of recent MT research focuses on pairs o...,0,train
3,P13-2073,A00-1002,An important challenge to statistical machine ...,Using examples of the transfer-based MT system...,Pivoting has been explored for closely related...,Many researchers have investigated the use of ...,Many different pivot strategies have been pres...,Pivoting has been explored for closely related...,Many researchers have investigated the use of ...,The first strategy is the sentence translation...,Pivoting has been explored for closely related...,Many researchers have investigated the use of ...,Many different pivot strategies have been pres...,0,train
4,W11-2602,A00-1002,This paper is about improving the quality of A...,Using examples of the transfer-based MT system...,"#REFR argued that for very close languages, e....","Moreover, Al-Sabbagh and Girju #OTHEREFR descr...",Zhang #OTHEREFR introduced a Cantonese-Mandari...,"#REFR argued that for very close languages, e....","Moreover, Al-Sabbagh and Girju #OTHEREFR descr...",Zhang #OTHEREFR introduced a Cantonese-Mandari...,"#REFR argued that for very close languages, e....","Moreover, Al-Sabbagh and Girju #OTHEREFR descr...",Zhang #OTHEREFR introduced a Cantonese-Mandari...,0,train


##### Get Human Labelled

In [47]:
cc_hl_df = cc_df[cc_df['human_labeled'] == 1]  # Replace 'column_name' with the name of the column you want to filter ondf.to_json('output.json', orient='records')  # Replace 'output.json' with the desired file name and path
cc_hl_df.to_json('../Data/human_labeled_cc.json', orient='records')  # Replace 'output.json' with the desired file name and path

In [48]:
cc_hl_df = cc_df[cc_df['human_labeled'] == 0] 


In [49]:
cc_hl_df['full_context']= cc_hl_df['text_before_explicit_citation'] + " "+ cc_hl_df['explicit_citation'] + " " +cc_hl_df['text_after_explicit_citation']

/var/folders/0q/frl6r__d72n6kwcf0721y37h0000gn/T/ipykernel_97178/283619694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cc_hl_df['full_context']= cc_hl_df['text_before_explicit_citation'] +" "+ cc_hl_df['explicit_citation'] + " " +cc_hl_df['text_after_explicit_citation']


# Model Training


In [ ]:
from transformers import RobertaTokenizer, RobertaForQuestionAnswering
from transformers import Trainer, TrainingArguments
import torch